In [4]:
import pandas as pd
import plotly.express as px
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
import pandas as pd
import plotly.express as px
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV
from haversine import haversine

In [16]:
df = pd.read_csv('with_nans_in_type_and_with_random_metro.csv')
df = pd.read_csv('bhvkrsjgrvfskdf.csv')
#df = df.dropna(subset = 'Price')
#df.type = df.type.fillna('monolith')
#df['Number of rooms'].fillna(0, inplace = True)
# df = df[df.Price < 50 * 10**6]
# df = df[df.Area < 500]
df = df.drop(columns = ['Unnamed: 0',# 'id',
                        'date'])

#------------------------------------------------------------------------
### METROS
def calculate_distance(row):
    point_x = (row['lat'], row['lon'])
    point_y = (row['lat_metro'], row['lon_metro'])
    return haversine(point_x, point_y, unit='km')

list_stations = ['Ховрино', 'Сокол', 'Остров мечты', 'Минская', 'Коммунарка', 'Бульвар Генерала Карбышева', 'Удельная', 'Ольховая', 'Орехово', 'Улица Горчакова', 'Улица Академика Королёва', 'Подрезково', 'Кратово', 'Гражданская', 'Коптево', 'Пенягино', 'Железнодорожная', 'Верхние Лихоборы', 'Москва-Товарная', 'Яхромская', 'Сухаревская', 'Фрунзенская', 'Люберцы', 'Проспект Вернадского', 'Беломорская', 'Речной вокзал', 'Одинцово', 'Улица 1905 года', 'Лесной Городок', 'Марк', 'Академическая', 'Жулебино', 'Плющево', 'Силикатная', 'Троицк', 'Лубянка', 'Марьина Роща', 'Ботанический сад', 'Десна', 'Медведково', 'Новодачная', 'Полежаевская', 'Фили', 'Университет', 'Сходня', 'Цветной бульвар', 'Тимирязевская', 'Сколково', 'Крылатское', 'Комсомольская', 'Парк Победы', 'Дубровка', 'Севастопольская', 'Домодедовская', 'Тульская', 'Угрешская', 'Зорге', 'Кусково', 'Баковка', 'Серпуховская', 'Воронцовская', 'Кутузовская', 'Химки', 'Грачёвская', 'Дегунино', 'Воробьёвы горы', 'Перерва', 'Летово', 'Каспийская', 'Беляево', 'Гольяново', 'Лобня', 'Отрадное', 'Ростокино', 'Курская', 'Боровское шоссе', 'Тропарёво', 'Алма-Атинская', 'Бутово', 'Черкизовская', 'Озёрная', 'Матвеевская', 'Студенческая', 'Прокшино', 'Есенинская', 'Опалиха', 'Фонвизинская', 'Мещерская', 'Реутов', 'Бунинская аллея', 'Бульвар Рокоссовского', 'Профсоюзная', 'Шелепиха', 'Кавказский бульвар', 'Каширская', 'Лианозово', 'Суворовская', 'Перово', 'Трубная', 'Соколиная гора', 'Третьяковская', 'Новоподрезково', 'Подольск', 'Говорово', 'Аннино', 'Нахимовский проспект', 'Кузнецкий мост', 'Лухмановская', 'Верхние котлы', 'Немчиновка', 'Боровицкая', 'Улица Академика Янгеля', 'Улица Старокачаловская', 'Деловой центр', 'Хорошёвская', 'Митино', 'Рижская', 'Мякинино', 'Зеленоград — Крюково', 'Первомайская', 'Чеховская', 'Щёлковская', 'Остафьево', 'Южный порт', 'Марксистская', 'Ватутинки', 'Локомотив', 'Ильинская', 'Тютчевская', 'Кокошкино', 'Никольское', 'Терехово ', 'Тургеневская', 'Депо', 'Нагорная', 'Лихоборы', 'Народное Ополчение', 'Бибирево', 'Быково', 'Зюзино', 'Нахабино', 'Щукинская', 'Октябрьское поле', 'Липецкая', 'Останкино', 'Новоясеневская', 'Ясенево', 'Звенигородская', 'Филатов Луг', 'Петровский Парк', 'Кантемировская', 'Новокузнецкая', 'Серп и Молот', 'Волгоградский проспект', 'Шереметьевская', 'Панки', 'Долгопрудная', 'Войковская', 'Новопеределкино', 'Ипподром', 'Китай-город', 'Люблино', 'Сортировочная', 'Окружная', 'Рассказовка', 'Южная', 'Спортивная', 'Серебряный Бор', 'Рязанский проспект', 'Вешняки', 'Петровско-Разумовская', 'Нагатинская', 'Киевская', 'Динамо', 'Толстопальцево', 'Томилино', 'Площадь Ильича', 'Потапово', 'Кропоткинская', 'Тушинская', 'Вавиловская', 'Славянский бульвар', 'Левобережная', 'Очаково', 'Сетунь', 'Раменское', 'Красный Строитель', 'Мнёвники', 'Смоленская', 'Белокаменная', 'Измайлово', 'Телецентр', 'Фирсановская', 'Кузьминки', 'Октябрьская', 'Мичуринец', 'Красные ворота', 'Новаторская', 'Щербинка', 'Пушкинская', 'Балтийская', 'Битцевский парк', 'Проспект Мира', 'Сходненская', 'Кленовый бульвар', 'ВДНХ', 'Чистые пруды', 'Покровское', 'Победа', 'Менделеевская', 'Отдых', 'Измайловская', 'Партизанская', 'Лермонтовский проспект', 'Площадь Гагарина', 'Алтуфьево', 'Владыкино', 'Ольгино', 'Кучино', 'Технопарк', 'Бирюлёво', 'Физтех', 'Битца', 'Бачуринская', 'Аэропорт', 'Бульвар Дмитрия Донского', 'Университет Дружбы Народов', 'Свиблово', 'Шаболовская', 'ЗИЛ', 'Беговая', 'Павшино', 'Кожуховская', 'Молжаниново', 'Охотный ряд', 'Улица Дмитриевского', 'Площадь трёх вокзалов', 'Ухтомская', 'Москворечье', 'Шипиловская', 'Лебедянская', 'Пролетарская', 'Новогиреево', 'Сокольники', 'Панфиловская', 'Краснопресненская', 'Театральная', 'Селигерская', 'Некрасовка', 'Трикотажная', 'Чухлинка', 'Таганская', 'Достоевская', 'Дмитровская', 'Маяковская', 'Тёплый Стан', 'Солнцево', 'Апрелевка', 'Савёловская', 'Аникеевка', 'Липовая Роща', 'Новокосино', 'Авиамоторная', 'Автозаводская', 'Москва-Сити', 'Курьяново', 'Рабочий посёлок', 'Волжская', 'Котельники', 'Каховская', 'Юго-Восточная', 'Римская', 'Водники', 'Волоколамская', 'Печатники', 'Стрешнево', 'Саларьево', 'Бутырская', 'Калужская', 'Пятницкое шоссе', 'Павелецкая', 'Варшавская', 'Солнечная', 'Румянцево', 'Улица Милашенкова', 'Фабричная', 'Поклонная', 'Хлебниково', 'Улица Сергея Эйзенштейна', 'Коньково', 'Красногвардейская', 'Лужники', 'Братиславская', 'Бульвар Адмирала Ушакова', 'Карамышевская', 'Калитники', 'Корниловская', 'Площадь Революции', 'Красногорская', 'Пионерская', 'Малаховка', 'Окская', 'Зябликово', 'Новослободская', 'Парк Культуры', 'Пражская', 'Бабушкинская', 'Добрынинская', 'Сретенский бульвар', 'Крестьянская застава', 'Чертановская', 'Пыхтино', 'ЦСКА', 'Косино', 'Красносельская', 'Баррикадная', 'Санино', 'Молодёжная', 'Давыдково', 'Крёкшино', 'Арбатская', 'Улица Скобелевская', 'Нижегородская', 'Чкаловская', 'Полянка', 'Текстильщики', 'Водный стадион', 'Генерала Тюленева', 'Выставочный центр', 'Аминьевская', 'Филёвский парк', 'Хорошёво', 'Библиотека им. Ленина', 'Выхино', 'Лефортово', 'Шоссе Энтузиастов', 'Новохохловская', 'Спартак', 'Крымская', 'Тверская', 'Моссельмаш', 'Кедровая', 'Электрозаводская', 'Алексеевская', 'Бескудниково', 'Кунцевская', 'Бауманская', 'Юго-Западная', 'Аэропорт Внуково', 'Белорусская', 'Красный Балтиец', 'Переделкино', 'Ленинский проспект', 'Салтыковская', 'Борисово', 'Красково', 'Александровский сад', 'Строгино', 'Новые Черёмушки', 'Царицыно', 'Марьино', 'Мичуринский проспект', 'Нагатинский Затон', 'Багратионовская', 'Стахановская', 'Андроновка', 'Преображенская площадь', 'Лесопарковая', 'Внуково', 'Ломоносовский проспект', 'Планерная', 'Семёновская', 'Сосенки', 'Раменки', 'Коломенская']
list_stations = [station.lower().replace(' ', '') for station in list_stations]
df = df[df['Metro station'].isin(list_stations)]

# metros = pd.read_csv('metros.csv', sep = ';', header = None)
# cols = ['Metro station', 'lat_metro', 'lon_metro']
# metros.columns = cols
# metros['Metro station'] = metros['Metro station'].str.lower()
# metros['Metro station'] = metros['Metro station'].str.replace(' ', '')
# df = pd.merge(df, metros, on = 'Metro station', how = 'left')
# df['distance_km'] = df.apply(calculate_distance, axis=1)

def calculate_distance_to_red_square(row):
    point_x = (row['lat'], row['lon'])
    point_y = (55.753544, 37.621202)
    return haversine(point_x, point_y, unit='km')
df['distance_to_red_square'] = df.apply(calculate_distance_to_red_square, axis=1)

def calculate_distance_to_moscow_city(row):
    point_x = (row['lat'], row['lon'])
    point_y = (55.74900, 37.53964)
    return haversine(point_x, point_y, unit='km')
df['distance_to_moscow_city'] = df.apply(calculate_distance_to_red_square, axis=1)

def calculate_distance_to_vdnh(row):
    point_x = (row['lat'], row['lon'])
    point_y = (55.831388, 37.629277)
    return haversine(point_x, point_y, unit='km')
df['distance_to_vdnh'] = df.apply(calculate_distance_to_red_square, axis=1)

def calculate_distance_to_svo(row):
    point_x = (row['lat'], row['lon'])
    point_y = (55.968071, 37.434100)
    return haversine(point_x, point_y, unit='km')
df['distance_to_svo'] = df.apply(calculate_distance_to_red_square, axis=1)
# --------------------------------------------------------------------------


df = df.dropna(subset = 'type') 
sd = df.drop(columns = ['Metro station'])
df = pd.get_dummies(df)

In [17]:
#Разбиение на таргет и признаки
X = df.drop(columns=['Price'])  # Признаки 
y = df['Price']  # Таргет
#разбиение на тестовую и обучающую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [18]:
params = {
    'objective': 'reg:squarederror',  # Функция потерь для регрессии
    'learning_rate': 0.15,  # Скорость обучения
    'max_depth': 7,  # Максимальная глубина деревьев
    'seed': 42  # Для воспроизводимости результатов
}
num_rounds = 3000  # Количество итераций (деревьев)
model = xgb.train(params, dtrain, num_rounds)

y_pred = model.predict(dtest)

# Оценка производительности модели
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
print('\nРЕЗУЛЬТАТЫ Gradient Boosting с XGBoost:', 
    '\nMSE: ', mse,
    '\nRMSE: ', rmse, 
    '\nMAE: ', mae, 
    '\nMAPE: ', mape,
    '\n\nRMSE in millions: ', (rmse/1000000).round(3), 'millions')


РЕЗУЛЬТАТЫ Gradient Boosting с XGBoost: 
MSE:  9585969154743.662 
RMSE:  3096121.6311288 
MAE:  1972689.7360707365 
MAPE:  12.10942227553453 

RMSE in millions:  3.096 millions


In [ ]:
param_grid = {
    'learning_rate': [0.05, 0.1, 0.15],  # Скорость обучения
    'max_depth': [8, 6, 10],  # Максимальная глубина деревьев
    'n_estimators': [1000, 3000, 6000],  # Количество деревьев
}

# Создаем модель XGBoost
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Задаем GridSearchCV с моделью и параметрами
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2)

# Запускаем поиск по сетке
grid_search.fit(X_train, y_train)

# Выводим лучшие параметры
print("Лучшие параметры:", grid_search.best_params_)

# Получаем лучшую модель
best_model = grid_search.best_estimator_

# Прогноз
y_pred = best_model.predict(X_test)

# Оценка производительности модели
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
print('\nРЕЗУЛЬТАТЫ Gradient Boosting с XGBoost и подобранными параметрами:', 
    '\nMSE: ', mse,
    '\nRMSE: ', rmse, 
    '\nMAE: ', mae, 
    '\nMAPE: ', mape,
    '\n\nRMSE in millions: ', (rmse/1000000).round(3), 'millions')

Fitting 3 folds for each of 36 candidates, totalling 108 fits
[CV] END .learning_rate=0.05, max_depth=5, n_estimators=1000; total time=   8.0s
[CV] END .learning_rate=0.05, max_depth=5, n_estimators=1000; total time=   8.1s
